In [ ]:
import os
import gradio as gr
from dotenv import load_dotenv
from openai import OpenAI

In [ ]:
# Load API Key
load_dotenv()
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

In [ ]:
# -------------------------------
# 1. System Prompt (Business Context)
# -------------------------------
system_message = """
You are Nova, an AI Sales & Solutions Consultant for Reallytics.ai a company specializing in building
custom AI chatbots, voice assistants, data dashboards, and automation solutions for businesses.
You are professional, insightful, and always focused on solving the user's business challenges.
First, try to understand their use case. Then suggest relevant solutions from our services with clear value propositions.
If the user is unsure, give them examples of how similar businesses have benefited from AI.
"""

MODEL = "gpt-4o-mini"

In [ ]:
# -------------------------------
# 2. Smart Chat Function (Streaming)
# -------------------------------
def chat(message, history):
    # Convert Gradio's chat history to OpenAI format
    history_messages = [{"role": h["role"], "content": h["content"]} for h in history]

    # Adjust system message based on context dynamically
    relevant_system_message = system_message
    if "price" in message.lower():
        relevant_system_message += (
            " If the user asks about pricing, explain that pricing depends on project complexity, "
            "but typical POCs start around $2,000 - $5,000, and full enterprise deployments scale beyond that."
        )
    if "integration" in message.lower():
        relevant_system_message += (
            " If integration is mentioned, reassure the user that our solutions are built to integrate seamlessly with CRMs, ERPs, or internal APIs."
        )

    # Compose final messages
    messages = [{"role": "system", "content": relevant_system_message}] + history_messages + [
        {"role": "user", "content": message}
    ]

    # Stream the response
    stream = client.chat.completions.create(
        model=MODEL,
        messages=messages,
        stream=True
    )

    response = ""
    for chunk in stream:
        response += chunk.choices[0].delta.content or ""
        yield response

In [ ]:
# -------------------------------
# 3. Gradio Chat UI
# -------------------------------
with gr.Blocks(title="AI Business Assistant") as demo:
    gr.Markdown("# AI Business Assistant\nYour intelligent sales and solution consultant, powered by OpenAI.")

    
gr.ChatInterface(
    fn=chat,
    type="messages",
    title="Business AI Consultant",
    description="Ask about automation, chatbots, dashboards, or voice AI Nova will help you discover the right solution."
).launch()
